In [2]:
# GPU 연결 확인 커맨드  (문제시 Edit-Notebook settings-Hardware accelerator를 GPU로 세팅-Save)
# %nvidia-smi

In [3]:
from IPython.display import clear_output

In [4]:
%pip install roboflow
%pip install ultralytics==8.0.20
clear_output()

In [5]:
import os
HOME = os.getcwd()
print(HOME)

/Users/leeshinhee/Documents/newdeal/project/CloudAiProject_team1_preprocessing


In [6]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.10.4 torch-2.3.0 CPU
Setup complete ✅ (8 CPUs, 16.0 GB RAM, 212.6/228.3 GB disk)


In [7]:
from ultralytics import YOLO
from IPython.display import display, Image

## pretrained 모델 예측 예시 

In [8]:
# # Inference with Pre-trained COCO Model
# %cd {HOME}
# ! yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

In [9]:
# %cd {HOME}
# Image(filename='runs/detect/predict/dog.jpeg')

In [10]:
# model = YOLO(f'{HOME}/yolov8n.pt')
# results = model.predict(source='https://media.roboflow.com/notebooks/examples/dog.jpeg', conf=0.25)

In [11]:
# print(results[0].boxes.xyxy)
# print(results[0].boxes.conf)
# print(results[0].boxes.cls)

## 내 데이터 가져오기 

In [12]:
from roboflow import Roboflow
import os

# dir 생성
HOME = os.getcwd()
datasets_dir = os.path.join(HOME, "datasets")
os.makedirs(datasets_dir, exist_ok=True)
os.chdir(datasets_dir)

In [ ]:
from dotenv import load_dotenv
load_dotenv()
API_KEY = os.getenv("ROBOFLOW_REF_MODEL_API")

In [13]:
rf = Roboflow(API_KEY)
project = rf.workspace('goai4thprojectobjectdetection').project("casting_data_goai4th")
dataset = project.version(6).download("yolov8")

os.listdir(dataset.location)


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.20, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Casting_data_goai4th-6 in yolov8:: 100%|██████████| 16048/16048 [00:01<00:00, 8877.38it/s] 


['README.roboflow.txt',
 'valid',
 'README.dataset.txt',
 'test',
 'data.yaml',
 'train']

In [14]:
print(dataset.location)

/Users/leeshinhee/Documents/newdeal/project/CloudAiProject_team1_preprocessing/datasets/Casting_data_goai4th-6


## Custom Training

In [15]:
%cd {HOME}

! yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=10 imgsz=320 plots=True

# CLI 공식문서 https://docs.ultralytics.com/usage/cfg/#train

/Users/leeshinhee/Documents/newdeal/project/CloudAiProject_team1_preprocessing
Ultralytics YOLOv8.0.20 🚀 Python-3.10.4 torch-2.3.0 CPU
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/Users/leeshinhee/Documents/newdeal/project/CloudAiProject_team1_preprocessing/datasets/Casting_data_goai4th-6/data.yaml, epochs=10, patience=50, batch=16, imgsz=320, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False

In [ ]:
!ls {HOME}/runs/detect/train/

args.yaml weights


In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=100)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=100)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=100)

## Validation

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

## Inference

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.5 iou=0.01 source={dataset.location}/test/images save=True

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict2/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")